In [6]:
# CDC Daily Census PM2.5 Particles 2011 - 2014
# Christian Granados
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import os

In [7]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# CDC dataset
air_df = pd.read_csv("/content/drive/MyDrive/Data/Daily_Census_Tract-Level_PM2.5_Concentrations__2011-2014.csv")

In [11]:
# FIPS -> City, State
fips_df = pd.read_excel("/content/drive/MyDrive/Data/US_FIPS_Codes.xls", skiprows = 1)

In [12]:
air_df.drop(columns = ["date"], inplace = True)

In [ ]:
air_df1 = air_df.groupby(by = ["statefips", "countyfips"]).agg([
    "mean",
    "median",
    lambda x: np.percentile(x, 75),
    lambda x: np.percentile(x, 85),
    lambda x: np.percentile(x, 90),
    lambda x: np.percentile(x, 95),
    lambda x: np.percentile(x, 99),
    lambda x: np.percentile(x, 99.5)
])

In [ ]:
air_df2 = air_df1.reset_index().droplevel(0, axis = 1)
air_df2.columns = ["statefips", "countyfips", "mean", "median", "p75", "p85", "p90", "p95", "p99", "p99.5"]
air_df2.drop(columns = ["statefips"], inplace = True)
air_df2["countyfips"] = air_df2["countyfips"].map(lambda s: str(s))

In [ ]:
fips_df1 = fips_df.copy(deep = True)
fips_df1["countyfips"] = fips_df1["FIPS State"].map(lambda n: str(n).strip()) + fips_df1["FIPS County"].map(lambda n : "0" * (3 - len(str(n))) + str(n))
fips_df1.drop(columns = ["FIPS State", "FIPS County"], inplace = True)

In [ ]:
# Used to get the 500 cities location and names
fhc_df = pd.read_csv("/content/drive/MyDrive/Data C102 F2023 - Final Project/Data/500_Cities__Chronic_obstructive_pulmonary_disease_among_adults_aged___18_years.csv")

In [ ]:
fhc_df1 = fhc_df[["StateDesc", "CityName"]].drop_duplicates(ignore_index = True)

In [ ]:
# Used to go from cities <-> counties
cc_df = pd.read_csv("/content/drive/MyDrive/Data C102 F2023 - Final Project/Data/us_cities.csv")

In [ ]:
cc_df1 = cc_df.copy(deep = True)[["STATE_NAME", "COUNTY", "CITY"]]

In [ ]:
fips_df2 = fips_df1.merge(
    cc_df1,
    how = "left",
    left_on = ["State", "County Name"],
    right_on = ["STATE_NAME", "COUNTY"]
    ).drop(
        columns = ["STATE_NAME", "COUNTY"]
    ).rename(
        {"CITY" : "City"},
        axis = 1
    )

In [ ]:
fips_df3 = fips_df2.merge(
    fhc_df1,
    how = "inner",
    left_on = ["State", "City"],
    right_on = ["StateDesc", "CityName"]
).drop(
    columns = ["State", "City"]
)

In [ ]:
fips_df4 = fips_df3.merge(
    air_df2,
    how = "inner",
    on = "countyfips"
)

In [ ]:
fips_df4

,County Name,countyfips,StateDesc,CityName,mean,median,p75,p85,p90,p95,p99,p99.5
0,Jefferson,1073,Alabama,Birmingham,11.815505,11.157021,14.518124,16.659414,17.894518,20.127802,24.995912,26.872805
1,Shelby,1117,Alabama,Birmingham,11.252897,10.515597,13.801060,15.920127,17.147858,18.967579,24.467754,25.959950
2,Madison,1089,Alabama,Huntsville,10.893989,10.314925,13.335756,15.249538,16.580532,18.389011,22.870915,24.449241
3,Mobile,1097,Alabama,Mobile,9.726570,9.092279,11.579182,13.290443,14.410183,16.502107,19.630501,21.298053
4,Montgomery,1101,Alabama,Montgomery,10.932706,10.310074,13.323652,15.184963,16.695852,18.565612,22.455000,24.705159
...,...,...,...,...,...,...,...,...,...,...,...,...
482,Milwaukee,55079,Wisconsin,Milwaukee,10.341980,9.206295,13.268079,15.890329,17.861441,20.681140,28.196422,30.744375
483,Outagamie,55087,Wisconsin,Appleton,9.325449,7.989818,12.096473,14.612607,16.265477,18.976858,27.075498,30.428618
484,Racine,55101,Wisconsin,Racine,10.332862,9.174307,13.274800,15.821566,17.606961,20.685289,28.131156,31.368446
485,Waukesha,55133,Wisconsin,Waukesha,10.240954,9.123153,13.072183,15.549006,17.525053,20.584124,26.836456,31.423809
